# From npy (좌표변환 (원점이동 및 단위거리 스케일링)) to Trainig Data

In [1]:
import numpy as np
import os
import sys
import glob
from utils import *

/usr/local/lib/python3.8/dist-packages/scipy/__init__.py:143: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.17.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#dtype   = get_dtypes('COMMON',False)
folder_data_processed1 = '/home/hyoyeonlee/bbs_project/data_processed/_1_coor_transformed/'

# task마다 훈련데이터로 사용하는 카메라 방향 정해져 있음.

- 측면 ('a'), 정면 ('e')중에 한 방향 데이터만 사용한다. 
- 따라서 하단링크 파일에 명시된 task-camDirection 따르면 됨. 

https://github.com/Deep-In-Sight/Kinect_BBS_demo/blob/main/bbsQt/constants.py


- 각 task마다 지정된 CAM_NAMES의 데이터만 추출해서 리스트화 == fnames_task : {task(int):fnames}

CAM_NAMES=

{1: "e",            2: "e",            3: "a",            4: "e",<br />
 5: "e",            6: "e",            7: "e",            8: "a",<br />
 9: "a",           10:"e",           11:"e",           12:"e", <br />
 13:"a",           14:"e"}

In [3]:
CAM_NAMES={
    1: "e",           
    2: "e",           
    3: "a",           
    4: "e",               
    5: "e",           
    6: "e",           
    7: "e",           
    8: "a",  
    9: "a",           
    10:"e",           
    11:"e",           
    12:"e",  
    13:"a",           
    14:"e"
            }
fnames_task = {}
for i in range(1,15):
    camDir = CAM_NAMES[i]
    fnames = glob.glob(folder_data_processed1+camDir+'/'+str(i)+'/*/*/*/*')
    fnames.sort()
    fnames_task[i]=fnames

# flow 이해

- nframe : 한 scene에서 데이터화 할 프레임 갯수 (등간격), 즉, 한 데이터당 frame갯수
- frame_skip : 한 scene에서 데이터의 갯수
- DATA : 크기 14인 array이고 각 element는 task별 모아놓은 데이터임.
- LABEL : 역시 크기 14인 array이고 각 element는 task별 모아놓은 데이터의 스코어 이다.

In [4]:
nframe = 10 #org에서는 6
frame_skip = 20 #한 scene에서 20개의 훈련데이터 셋을 만들겠다.
DATA = []
LABEL = []
feature = []

for i in range( len(fnames_task)):
    fnames = fnames_task[i+1]
    data=[]
    label=[]
    for j in range(len(fnames)):
        fname = fnames[j]
        f_scene = np.load(fname)
        f_info = fname.split('/')[-1].split('.')[0].split('_')
        camDir=f_info[0]
        sceneID = f_info[1]
        task = int(f_info[2])
        score = int(f_info[3])
        for k in range(frame_skip):
            sub = smoothed_frame_N(f_scene,nframe=nframe,shift =k)
            if j==k==0:
                feature, rav_sub = ravel_rec(sub,return_feature=True)
            else:
                rav_sub = ravel_rec(sub)
            if True in np.isnan(rav_sub):
                continue
            else:
                data.append(rav_sub)
                label.append(score)
    DATA.append(data)
    LABEL.append(label)
len(DATA)
    
    

14

In [5]:
for i in range(len(DATA)):
    print(len(DATA[i]))

8383
8380
8320
8369
8352
8340
8380
8227
8340
8358
8358
8360
8340
8380


# 각 task마다 해당 카메라방향, mainID의 skeleton정보 취합, npy로 저장

In [6]:
for i in range(14):
    fn = f"/home/hyoyeonlee/bbs_project/data_processed/_2_training/task{i+1:02}_"
    np.save(fn+"data.npy",np.stack(DATA[i]))
    np.save(fn+"label.npy",np.stack(LABEL[i]))